In [1]:
#pip install polytope


In [2]:
#pip install pypoman

In [3]:
# pip install polytope
# pip install pypoman

# import math
# import numpy as np
# from sympy.geometry import Point
# import matplotlib.pyplot as plt
# from polytope import polytope
# from polytope.polytope import enumerate_integral_points, qhull, box2poly
# from scipy.spatial import ConvexHull

Let's construct a reflexive 2D polytope (taken from the book "The Calabi–Yau Landscape", page 44)

In [4]:
#V = np.array([[0, -1], [-1, 0], [-1, 2], [1, 0], [1, -1]])


Pypoman provides a form $0\le -Ax+b$

In [5]:


# #print(compute_polytope_vertices(A, b))
# print(ConvexHull(V).npoints)

# GA TEST

In [6]:
import math
import numpy as np
from sympy.geometry import Point
import matplotlib.pyplot as plt
# from polytope import polytope
# from polytope.polytope import enumerate_integral_points, qhull, box2poly
from scipy.spatial import ConvexHull
import random
from random import randint
import copy
from fractions import Fraction
import cdd
from polytope.polytope import qhull, enumerate_integral_points, ABS_TOL


class Population:
    def __init__(self, number_of_generations, survival_rate, crossover_type, crossover_parent_choosing_type, chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point):
        #self.generation_0 = Generation(chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point)
        
        self.number_of_generations = number_of_generations
        self.all_generations = []
        
        self.all_generations.append(Generation(chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point))
        
        for curr_generation_number in range(0, number_of_generations-1):
            if curr_generation_number % 10 == 0:
                print("curr Generation count: " + str(curr_generation_number))
                
            self.copy_prev_Generation_chrom_list = copy.deepcopy(self.all_generations[curr_generation_number].get_chrom_list())
            self.copy_prev_Generation_chrom_list = self.copy_prev_Generation_chrom_list[:int(len(self.copy_prev_Generation_chrom_list)/2)]
            parents = []
            while len(self.copy_prev_Generation_chrom_list) < len(self.all_generations[0].get_chrom_list()):
                parents = random.sample(self.copy_prev_Generation_chrom_list, 2)
                
                crossover_chromosom = Chromosom.apply_crossover(parents[0], parents[1], crossover_type)
                
                if crossover_chromosom.get_is_valid():
                    self.copy_prev_Generation_chrom_list.append(crossover_chromosom)
                
            self.copy_prev_Generation_chrom_list.sort(key=lambda x: x.fitness, reverse=True)
            self.all_generations.append(Generation(self.copy_prev_Generation_chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point))

    def get_all_generations(self):
        return self.all_generations


class Generation:
    num_points_per_polytope = 0
    num_chroms_per_Generation = 0
    num_dimensions = 0 # 3
    min_value_per_point = 0  # -3
    max_value_per_point = 0  # 3
    
    
    def __init__(self, chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point):
        
        Generation.num_points_per_polytope = num_points_per_polytope
        Generation.num_chroms_per_Generation = num_chroms_per_Generation
        Generation.num_dimensions = num_dimensions # 3
        Generation.min_value_per_point = min_value_per_point  # -3
        Generation.max_value_per_point = max_value_per_point  # 3
        
        if chrom_list == None:
            self.chrom_list = self.gen_first_generation(num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point)
        else:
            self.chrom_list = chrom_list

        self.generation_fitness_list = self.set_fitness_value_list(self.chrom_list)

    def set_fitness_value_list(self, chrom_list):
        fitness = []
        for chrom in chrom_list:
            fitness.append(chrom.get_fitness())

        #print(fitness)
        return(fitness)

    def get_chrom_list(self):
        return self.chrom_list
    
    def get_dimension():
        return Generation.num_dimensions

    def get_min_value_per_point():
        return Generation.min_value_per_point
    
    def get_max_value_per_point():
        return Generation.max_value_per_point

    def get_fitness_values_list(self):
        return self.generation_fitness_list

    def gen_first_generation(self, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point):
        #num_points_per_polytope = 5
        #num_chroms_per_Generation = 10
        #num_dimensions = num_dimensions

        polytope_points = []
        generation_polytopes = []

        #for j in range(0, num_chroms_per_Generation):
        while len(generation_polytopes) < num_chroms_per_Generation:
            #chromosome_test_points=[]
            polytope_points = []
            for i in range(0,num_points_per_polytope):
                polytope_points.append(random.sample(range(min_value_per_point, max_value_per_point), num_dimensions))
            chromosome_polytope_points = np.array(polytope_points)
            #print(polytope_points)
            chromosom_test = Chromosom(chromosome_polytope_points)

            if chromosom_test.get_is_valid():
                generation_polytopes.append(chromosom_test)
        
        generation_polytopes.sort(key=lambda x: x.fitness, reverse=True)
        #print("generation done")
        return generation_polytopes

class Chromosom:
    chrom_count = 0
    reflexive_politopes = {}

    def __init__(self, points):
        chromosome_polytope_points = points
        #print("pre try")
        #print(Generation.num_dimensions)
        self.is_valid = False


        #result = None
        #while result is None:
        try:
            internal_points = []
            #print("###############################################")
            Chromosom.chrom_count = Chromosom.chrom_count + 1
            #print(Chromosom.chrom_count)
            
            self.gen_list = chromosome_polytope_points
            self.fitness = self.calc_fitness(chromosome_polytope_points)
            #print(True)
            self.is_valid = True

            if self.fitness == 0: 
                print(True)
                polytope_volume = ConvexHull(chromosome_polytope_points).volume #qhull(chromosome_polytope_points).volume
                Chromosom.reflexive_politopes[polytope_volume] = ConvexHull(self.gen_list)
            #result = 1
        except:
            self.is_valid = False
    
    def get_is_valid(self):
        return self.is_valid
    
    def get_gen_list(self):
        return self.gen_list
    
    def get_fitness(self):
        return self.fitness

    def get_gen_list(self):
        return self.gen_list

    #code based on https://github.com/stephane-caron/pypoman/blob/master/pypoman/duality.py
    def compute_distances(self, vertices: np.ndarray) -> np.ndarray:
        #print("yyyyyyyyyyyyyyyyyyyyyyyyyyyyy")
        
        V = np.vstack(vertices)
        t = np.ones((V.shape[0], 1))  # first column is 1 for vertices
        tV = np.hstack([t, V])
        mat = cdd.Matrix(tV, number_type="fraction")
        mat.rep_type = cdd.RepType.GENERATOR
        P = cdd.Polyhedron(mat)
        bA = np.array(P.get_inequalities())
        if bA.shape == (0,):  # bA == []
            return bA
        # the polyhedron is given by b + A x >= 0 where bA = [b|A]
        b = bA[:, 0]
        A = -np.array(bA[:, 1:])

        for i, row in enumerate(A):
            denominators = []
            has_fraction = False
            for j in row:
                if type(j) == Fraction:
                    has_fraction = True
                    denominators.append(j.denominator)    
            if has_fraction:
                b[i] *= np.lcm.reduce(denominators)
        return b

    def calc_fitness(self, vertices):
        
        integral_points = enumerate_integral_points(qhull(vertices))
        integral_points = integral_points.transpose()
        integral_points = integral_points.astype(int)
        ip_count = len(integral_points)
        #print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
        #print(np.array(vertices))
        distances = self.compute_distances(np.array(vertices))
        result = 0
        if ip_count > 1:
            result -= 1
        for d in distances:
            result -= abs(d-1)
        
        return result

    def apply_crossover(chromosom_1, chromosom_2, crossover_type):
        #print("crossover")
        if chromosom_1.get_fitness() > chromosom_2.get_fitness():
            chromosom_1_points = chromosom_1.get_gen_list()
            chromosom_2_points = chromosom_2.get_gen_list()
        else:
            chromosom_2_points = chromosom_1.get_gen_list()
            chromosom_1_points = chromosom_2.get_gen_list()
        
        crossover_type = crossover_type

        new_chromosom_gen_list = []
        if crossover_type == 1:
            
            if len(chromosom_1_points)>len(chromosom_2_points):
                length_new_chromosom = len(chromosom_2_points)
                random_crossover_point = random.randint(1,len(chromosom_2_points)-1)
            else:
                length_new_chromosom = len(chromosom_2_points)
                random_crossover_point = random.randint(1,len(chromosom_2_points)-1)

            #new_gen = []
            for i in range(0,length_new_chromosom):
                
                if random.random() < 0.8:
                    if i < random_crossover_point:
                        new_gen = []
                        for j in chromosom_1_points[i]:
                            new_gen.append(j)#[chromosom_1_points[0],chromosom_1_points[1]])
                    else:
                        new_gen = []
                        for j in chromosom_2_points[i]:
                            new_gen.append(j)

                    new_chromosom_gen_list.append(new_gen)
                    #print("random_gen_test")
                else:
                    new_chromosom_gen_list.append(random.sample(range(Generation.min_value_per_point, Generation.max_value_per_point), Generation.num_dimensions))
                    #print("random_gen_test")

        new_chromosom = Chromosom(np.array(new_chromosom_gen_list))

        return new_chromosom


In [10]:
# Population Test:

# test_population = Population(number_of_generations, survival_rate, crossover_type, crossover_parent_choosing_type, chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point)
number_of_generations = 100
survival_rate = None
crossover_type = 1
crossover_parent_choosing_type = None
chrom_list = None
num_points_per_polytope = 7
num_chroms_per_Generation = 50
num_dimensions = 5
min_value_per_point = -5
max_value_per_point = 5

test_population = Population(number_of_generations, survival_rate, crossover_type, crossover_parent_choosing_type, chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point)



curr Generation count: 0
curr Generation count: 10
curr Generation count: 20
curr Generation count: 30
curr Generation count: 40
curr Generation count: 50
curr Generation count: 60
curr Generation count: 70
curr Generation count: 80
curr Generation count: 90


In [8]:
# V = np.array([[1, 0, -2], [-2, -3, 2], [-1, -2, 1], [1, 2, -1], [1, 2, 0]])

# print(Chromosom.calc_fitness(V))

In [11]:
test_population_generations = test_population.get_all_generations()
test_population_generations_chroms = test_population_generations[1].get_chrom_list()
#print(test_population_generations_chroms)

all_fitness = []
for generation in test_population.get_all_generations():
    #fitness = [generation.get_fitness_values_list()]
    # for chrom in generation.get_chrom_list():
    #     fitness.append(chrom.get_fitness())

    print(generation.get_fitness_values_list())

    all_fitness.append(generation.get_fitness_values_list())
    #print(generation.get_fitness_values_list())
    #all_fitness.append(fitness)
#print(all_fitness)
print(len(all_fitness))
#print(all_fitness)

[Fraction(-1466, 1), Fraction(-1690, 1), Fraction(-2152, 1), Fraction(-2846, 1), Fraction(-2993, 1), Fraction(-3157, 1), Fraction(-3354, 1), Fraction(-3659, 1), Fraction(-4331, 1), Fraction(-4367, 1), Fraction(-4614, 1), Fraction(-4639, 1), Fraction(-4679, 1), Fraction(-4691, 1), Fraction(-5205, 1), Fraction(-5397, 1), Fraction(-5592, 1), Fraction(-5804, 1), Fraction(-5913, 1), Fraction(-6741, 1), Fraction(-6801, 1), Fraction(-7155, 1), Fraction(-8762, 1), Fraction(-9120, 1), Fraction(-9206, 1), Fraction(-9206, 1), Fraction(-9697, 1), Fraction(-9729, 1), Fraction(-10127, 1), Fraction(-10843, 1), Fraction(-10879, 1), Fraction(-12014, 1), Fraction(-12027, 1), Fraction(-12562, 1), Fraction(-13501, 1), Fraction(-13853, 1), Fraction(-14258, 1), Fraction(-14310, 1), Fraction(-14396, 1), Fraction(-15904, 1), Fraction(-16433, 1), Fraction(-17710, 1), Fraction(-18400, 1), Fraction(-19031, 1), Fraction(-19069, 1), Fraction(-19299, 1), Fraction(-21335, 1), Fraction(-22474, 1), Fraction(-22568, 1)

In [ ]:
test_population_generations = test_population.get_all_generations()
test_population_generations_chroms = test_population_generations[2].get_chrom_list()
#print(test_population_generations_chroms)

print(test_population_generations_chroms[0].get_gen_list())
print(test_population_generations_chroms[0].get_fitness())

# all_fitness = []
# for generation in test_population.get_all_generations():
#     fitness = [generation.get_fitness_values_list()]
#     for chrom in generation.get_chrom_list():
#         fitness.append(chrom.get_fitness())

#     #print(generation.get_fitness_values_list())

#     all_fitness.append(generation.get_fitness_values_list())
#     #print(generation.get_fitness_values_list())
#     #all_fitness.append(fitness)
# #print(all_fitness)
# print(len(all_fitness))
# #print(all_fitness)

[[-5 -4  3  1  2]
 [-4 -1 -3  1  0]
 [-2 -3  3  2  0]
 [-1 -3  4  2  1]
 [ 4  1 -3  3 -4]
 [-4 -1 -2  1 -5]
 [-4 -3  3  0  1]]
-425


In [12]:
chrom_list = test_population_generations[len(test_population.get_all_generations())-1].get_chrom_list()

test_population2 = Population(number_of_generations, survival_rate, crossover_type, crossover_parent_choosing_type, chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point)

curr Generation count: 0
curr Generation count: 10
curr Generation count: 20
curr Generation count: 30
curr Generation count: 40
curr Generation count: 50
curr Generation count: 60
curr Generation count: 70
curr Generation count: 80
curr Generation count: 90


In [13]:
test_population_generations2 = test_population2.get_all_generations()
test_population_generations_chroms2 = test_population_generations2[1].get_chrom_list()
#print(test_population_generations_chroms)

all_fitness = []
for generation in test_population2.get_all_generations():
    #fitness = [generation.get_fitness_values_list()]
    # for chrom in generation.get_chrom_list():
    #     fitness.append(chrom.get_fitness())

    print(generation.get_fitness_values_list())

    all_fitness.append(generation.get_fitness_values_list())
    #print(generation.get_fitness_values_list())
    #all_fitness.append(fitness)
#print(all_fitness)
print(len(all_fitness))
#print(all_fitness)

[Fraction(-11, 1), Fraction(-11, 1), Fraction(-11, 1), Fraction(-11, 1), Fraction(-11, 1), Fraction(-11, 1), Fraction(-11, 1), Fraction(-11, 1), Fraction(-11, 1), Fraction(-11, 1), Fraction(-11, 1), Fraction(-11, 1), Fraction(-11, 1), Fraction(-11, 1), Fraction(-18, 1), Fraction(-18, 1), Fraction(-18, 1), Fraction(-18, 1), Fraction(-18, 1), Fraction(-18, 1), Fraction(-18, 1), Fraction(-18, 1), Fraction(-18, 1), Fraction(-18, 1), Fraction(-18, 1), Fraction(-18, 1), Fraction(-18, 1), Fraction(-26, 1), Fraction(-88, 1), Fraction(-101, 1), Fraction(-129, 1), Fraction(-132, 1), Fraction(-249, 1), Fraction(-262, 1), Fraction(-385, 1), Fraction(-492, 1), Fraction(-598, 1), Fraction(-739, 1), Fraction(-801, 1), Fraction(-920, 1), Fraction(-988, 1), Fraction(-1157, 1), Fraction(-1378, 1), Fraction(-1680, 1), Fraction(-2328, 1), Fraction(-2333, 1), Fraction(-2645, 1), Fraction(-3599, 1), Fraction(-6175, 1), Fraction(-6193, 1)]
[Fraction(-11, 1), Fraction(-11, 1), Fraction(-11, 1), Fraction(-11, 

In [14]:
chrom_list = test_population_generations2[len(test_population2.get_all_generations())-1].get_chrom_list()

test_population3 = Population(number_of_generations, survival_rate, crossover_type, crossover_parent_choosing_type, chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point)

curr Generation count: 0
curr Generation count: 10
curr Generation count: 20
curr Generation count: 30
curr Generation count: 40
curr Generation count: 50
curr Generation count: 60
curr Generation count: 70
True
True
True
True
True
True
True
True
True
True
curr Generation count: 80
True
True
True
True
True
True
True
True
True
True
curr Generation count: 90
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [15]:
test_population_generations3 = test_population3.get_all_generations()
#test_population_generations_chroms2 = test_population_generations2[999].get_chrom_list()
#print(test_population_generations_chroms)

all_fitness = []
for generation in test_population3.get_all_generations():
    #fitness = [generation.get_fitness_values_list()]
    # for chrom in generation.get_chrom_list():
    #     fitness.append(chrom.get_fitness())

    print(generation.get_fitness_values_list())

    all_fitness.append(generation.get_fitness_values_list())
    #print(generation.get_fitness_values_list())
    #all_fitness.append(fitness)
#print(all_fitness)
print(len(all_fitness))
#print(all_fitness)

[-3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, Fraction(-37, 1), Fraction(-69, 1), Fraction(-104, 1), Fraction(-109, 1), Fraction(-210, 1), Fraction(-213, 1), Fraction(-311, 1), Fraction(-356, 1), Fraction(-370, 1), Fraction(-407, 1), Fraction(-457, 1), Fraction(-519, 1), Fraction(-528, 1), Fraction(-820, 1), Fraction(-863, 1), Fraction(-912, 1), Fraction(-947, 1), Fraction(-1220, 1), Fraction(-1413, 1), Fraction(-3556, 1), Fraction(-3986, 1), Fraction(-4513, 1), Fraction(-5953, 1)]
[-3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, Fraction(-31, 1), Fraction(-49, 1), Fraction(-147, 1), Fraction(-173, 1), Fraction(-249, 1), Fraction(-307, 1), Fraction(-353, 1), Fraction(-367, 1), Fraction(-402, 1), Fraction(-424, 1), Fraction(-467, 1), Fraction(-468, 1), Fraction(-663, 1), Fraction(-811, 1), Fraction(-965, 1), Fraction(-1351, 1), Fraction(-1413, 1), Fr

In [16]:
# a = [1,2,3]
# b = a.copy()
# #b = []
# #b.append(a[0])
# #b.append(a[1])
# a[1] = 5

# print(a)
# print(b)

for i in Chromosom.reflexive_politopes:
    print(i)
    print(Chromosom.reflexive_politopes[i].points)

print(Chromosom.reflexive_politopes)
print(len(Chromosom.reflexive_politopes))

0.5999999999999998
[[-2.  3.  1. -3.  0.]
 [ 1. -2. -1.  2.  0.]
 [ 4. -4.  1.  2. -2.]
 [ 1.  2.  3.  0. -1.]
 [-3.  3. -1. -2.  2.]
 [ 1.  2.  3. -3. -1.]
 [-2. -1. -3.  0.  2.]]
{0.5999999999999998: <scipy.spatial._qhull.ConvexHull object at 0x000001F08AEED6D0>}
1


In [ ]:
# Crossover test - to be checked

# # #def apply_crossover(chromosom_1, chromosom_2, crossover_type):
# chromosom_1_points = test_polytopes[1].get_gen_list()
# chromosom_2_points = test_polytopes[0].get_gen_list()
# # crossover_type = 1

# # new_chromosom_gen_list = []
# # if crossover_type == 1:
    

# #     if len(chromosom_1_points)>len(chromosom_2_points):
# #         length_new_chromosom = len(chromosom_2_points)
# #         random_crossover_point = random.randint(1,len(chromosom_2_points)-1)
# #     else:
# #         length_new_chromosom = len(chromosom_2_points)
# #         random_crossover_point = random.randint(1,len(chromosom_2_points)-1)

# #     #new_gen = []
# #     for i in range(0,length_new_chromosom):
# #         if i <= random_crossover_point:
# #             new_gen = []
# #             for j in chromosom_1_points[i]:
# #                 new_gen.append(j)#[chromosom_1_points[0],chromosom_1_points[1]])
# #         else:
# #             new_gen = []
# #             for j in chromosom_2_points[i]:
# #                 new_gen.append(j)

# #         new_chromosom_gen_list.append(new_gen)

# # new_chromosom = Chromosom(np.array(new_chromosom_gen_list))
# #return new_chromosom

# new_chromosom = Chromosom.apply_crossover(test_polytopes[1], test_polytopes[0], 1)

# #print(random_crossover_point)
# print(chromosom_1_points)
# print(chromosom_2_points)
# print(new_chromosom.get_gen_list())
# print(new_chromosom.get_fitness())


In [ ]:
# import numpy as np
# import polytope

# # halfspace representation computed using `polytope`
# # from the vertex representation given in the question
# vertices = np.array([[0, -1], [-1, 0], [-1, 2], [1, 0], [1, -1]])
# poly = polytope.qhull(vertices)
# # first halfspace representation
# A = np.array([
#     [0, -1],
#     [-1, 0],
#     [-1, -1],
#     [1, 1],
#     [1, 0]])
# b = np.array([1, 1, 1, 1, 1])
# question_poly_1 = polytope.Polytope(A, b)
# # second halfspace representation
# A = np.array([
#     [-0.70711, -0.70711],
#     [-1, -0],
#     [0.70711, 0.70711],
#     [0, -1],
#     [1, 0]])
# b = np.array([0.70711, 1, 0.70711, 1, 1])

# question_poly_2 = polytope.Polytope(A, b)
# # check that all the above halfspace representations
# # represent the same polytope
# assert poly == question_poly_1, (poly, question_poly_1)
# assert poly == question_poly_2, (poly, question_poly_2)

In [ ]:
# inner_point = np.array([0,0]) # in this case it is the origin
# fac1 = polytope.quickhull.Facet([[0, -1], [-1, 0]])
# fac2 = polytope.quickhull.Facet([[-1, 2], [1, 0]])
# fac3 = polytope.quickhull.Facet([[1, -1], [0, -1]])

# d1 = polytope.quickhull.distance(inner_point, fac1)
# d2 = polytope.quickhull.distance(inner_point, fac2)
# d3 = polytope.quickhull.distance(inner_point, fac3)
# print(d1)
# print(d2)
# print(d3)

Let's visualize our 2D polytope. For this purpose we use the [Polytope Library](https://github.com/tulip-control/polytope)

In [ ]:
# project into 2D



In [ ]:

# print(polytope.polytope.cheby_ball(P))

# min_V = np.min(V, axis=0) #min val of each cols
# min_x = min_V[0]
# min_y = min_V[1]
# max_V = np.max(V, axis=0) #max val of each cols
# max_x = max_V[0]
# max_y = max_V[1]

# ax = P.plot(linestyle="solid", linewidth=1, color="lightblue", edgecolor="red")
# ax.set_xlim([min_x-1, max_x+1])
# ax.set_ylim([min_y-1, max_y+1])
# plt.show()